In [1]:
%load_ext autoreload
%autoreload 2

import re
import nltk
import numpy as np
import pandas as pd
import os
from pronounResolution import *
from relationExtract import *

## Prepare Dataset

In [2]:
# get annotation files
files = [x for x in os.listdir('prep_scripts') if '_gapi' in x]

# read first annotation
df = pd.read_csv('prep_scripts/' + files[0])[['speaker', 'dialogue', 'sentences', 'sentiment', 'entities', 'tokens']]

# evaluate strings for lists/dicts of tokens, sentiment, entities
df['tokens'] = df['tokens'].apply(lambda x: eval(x))
df['sentiment'] = df['sentiment'].apply(lambda x: eval(x))
df['entities'] = df['entities'].apply(lambda x: eval(x))
df.head(10)

,speaker,dialogue,sentences,sentiment,entities,tokens
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{u'score': 0.3, u'magnitude': 0.3}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 0, u'begin': 0, u'pos': u'NUM', u'..."
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{u'score': 0.1, u'magnitude': 0.1}","[{u'type': u'WORK_OF_ART', u'meta': {}, u'sali...","[{u'index': 0, u'begin': 0, u'pos': u'NOUN', u..."
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{u'score': -0.6, u'magnitude': 0.6}",[],"[{u'index': 3, u'begin': 0, u'pos': u'PRON', u..."
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{u'score': -0.1, u'magnitude': 1}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 2, u'begin': 0, u'pos': u'X', u'la..."
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{u'score': 0.4, u'magnitude': 0.4}","[{u'type': u'OTHER', u'meta': {}, u'salience':...","[{u'index': 1, u'begin': 0, u'pos': u'PRON', u..."
5,Peggy Carter,Tell me that isn't what I think it is.,"[{'content': u""Tell me that isn't what I think...","{u'score': -0.6, u'magnitude': 0.6}",[],"[{u'index': 0, u'begin': 0, u'pos': u'VERB', u..."
6,Dr. Hank Pym,"It depends, if you think it's a poor attempt t...","[{'content': u""It depends, if you think it's a...","{u'score': -0.5, u'magnitude': 1}","[{u'type': u'OTHER', u'meta': {}, u'salience':...","[{u'index': 1, u'begin': 0, u'pos': u'PRON', u..."
7,Mitchell Carson,You were instructed to go to Russia. May I rem...,[{'content': u'You were instructed to go to Ru...,"{u'score': 0, u'magnitude': 0.7}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 2, u'begin': 0, u'pos': u'PRON', u..."
8,Dr. Hank Pym,I'm a scientist.,"[{'content': u""I'm a scientist."", 'begin': 0, ...","{u'score': 0.3, u'magnitude': 0.3}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 1, u'begin': 0, u'pos': u'PRON', u..."
9,Howard Stark,Then act like one. The Pym Particle is the mos...,"[{'content': u'Then act like one.', 'begin': 0...","{u'score': 0.1, u'magnitude': 1.2}","[{u'type': u'OTHER', u'meta': {}, u'salience':...","[{u'index': 1, u'begin': 0, u'pos': u'ADV', u'..."


In [3]:
cList = list(df.speaker.unique())
df['total_sent'] = df['sentiment'].apply(lambda x: x['score'] * x['magnitude'])
cDict = dict(df.groupby('speaker').total_sent.sum())
df.shift().head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{u'score': 0.3, u'magnitude': 0.3}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 0, u'begin': 0, u'pos': u'NUM', u'...",0.09
2,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{u'score': 0.1, u'magnitude': 0.1}","[{u'type': u'WORK_OF_ART', u'meta': {}, u'sali...","[{u'index': 0, u'begin': 0, u'pos': u'NOUN', u...",0.01
3,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{u'score': -0.6, u'magnitude': 0.6}",[],"[{u'index': 3, u'begin': 0, u'pos': u'PRON', u...",-0.36
4,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{u'score': -0.1, u'magnitude': 1}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 2, u'begin': 0, u'pos': u'X', u'la...",-0.10


In [4]:
df.loc[436]

speaker                                            Dr. Hank Pym
dialogue                    She doesn't think that we need you.
sentences     [{'content': u"She doesn't think that we need ...
sentiment                        {u'score': 0, u'magnitude': 0}
entities                                                     []
tokens        [{u'index': 3, u'begin': 0, u'pos': u'PRON', u...
total_sent                                                    0
Name: 436, dtype: object

## Task 1: Pronoun Resolution

In [5]:
# apply model to each line of dialogue, update tokens and character flag
df['tokens'], df['hasChar'] = zip(*df.apply(lambda x: pronResolution_base(cList, x), axis=1).values)
df.head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent,hasChar
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{u'score': 0.3, u'magnitude': 0.3}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 0, u'begin': 0, u'pos': u'NUM', u'...",0.09,False
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{u'score': 0.1, u'magnitude': 0.1}","[{u'type': u'WORK_OF_ART', u'meta': {}, u'sali...","[{u'index': 0, u'begin': 0, u'pos': u'NOUN', u...",0.01,False
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{u'score': -0.6, u'magnitude': 0.6}",[],"[{u'index': 3, u'begin': 0, u'pos': u'PRON', u...",-0.36,True
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{u'score': -0.1, u'magnitude': 1}","[{u'type': u'PERSON', u'meta': {}, u'salience'...","[{u'index': 2, u'begin': 0, u'pos': u'X', u'la...",-0.10,True
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{u'score': 0.4, u'magnitude': 0.4}","[{u'type': u'OTHER', u'meta': {}, u'salience':...","[{u'index': 1, u'begin': 0, u'pos': u'PRON', u...",0.16,True


In [ ]:
#
pronEval([df, df], numExamples=3)


******** line 517 ********
515. Scott Lang:
You know, I think this regulator is holding me back.

516. Dr. Hank Pym:
Do not screw with the regulator. If that regulator is compromised you would go sub-atomic.

=> 517. Scott Lang:
=> What does that mean?

518. Dr. Hank Pym:
It means that you would enter a quantum realm.

519. Scott Lang:
What does that mean?

******** test model 1: line 517 ********
1 pronouns resolved
1. What => ['Dr. Hank Pym']

how many are correctly identified? 0

******** line 517 ********
515. Scott Lang:
You know, I think this regulator is holding me back.

516. Dr. Hank Pym:
Do not screw with the regulator. If that regulator is compromised you would go sub-atomic.

=> 517. Scott Lang:
=> What does that mean?

518. Dr. Hank Pym:
It means that you would enter a quantum realm.

519. Scott Lang:
What does that mean?

******** test model 2: line 517 ********
1 pronouns resolved
1. What => ['Dr. Hank Pym']

how many are correctly identified? 0

******** line 537 *****

## Task 2: Relation Extraction

In [ ]:
df['relations'] = df.tokens.apply(lambda x:simpleRE(x))

In [ ]:
df[df.relations.notnull()].head().relations.values[1]

In [ ]:
df.head().tokens.apply(lambda x:simpleRE(x))